In [15]:
import pickle
import librosa
import numpy as np

In [64]:
# load the model 
filename = 'trained_model.sav'
load_model =pickle.load(open(filename, 'rb'))
load_model.weights

[<tf.Variable 'conv2d_12_2/kernel:0' shape=(5, 5, 1, 24) dtype=float32_ref>,
 <tf.Variable 'conv2d_12_2/bias:0' shape=(24,) dtype=float32_ref>,
 <tf.Variable 'conv2d_13_2/kernel:0' shape=(5, 5, 24, 48) dtype=float32_ref>,
 <tf.Variable 'conv2d_13_2/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'conv2d_14_2/kernel:0' shape=(5, 5, 48, 48) dtype=float32_ref>,
 <tf.Variable 'conv2d_14_2/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'dense_7_2/kernel:0' shape=(2400, 64) dtype=float32_ref>,
 <tf.Variable 'dense_7_2/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense_8_2/kernel:0' shape=(64, 5) dtype=float32_ref>,
 <tf.Variable 'dense_8_2/bias:0' shape=(5,) dtype=float32_ref>]

In [79]:
# Examples 
y, sr = librosa.load('Datasets/Drinking/474035__petebarry__fx-sipping-liquid-01.wav', duration=2.97)
ps1 = librosa.feature.melspectrogram(y=y, sr=sr)

y, sr = librosa.load('Datasets/Shaking/1–3 Gloucester Road 109.m4a', duration=2.97)
ps2 = librosa.feature.melspectrogram(y=y, sr=sr)

y, sr = librosa.load('Datasets/Pouring/79188__nathan-lomeli__gurgling-drinking-fountain.wav', duration=2.97)
ps3 = librosa.feature.melspectrogram(y=y, sr=sr)


In [80]:
D = [] # Dataset

D.append( (ps1, 1) ) # choose random number, which is used for validation 
D.append( (ps2, 4) )
D.append( (ps3, 3) )

In [82]:
X_test, y_test = zip(*D)
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

In [83]:
# load the encoder
with open("encoderX.txt", "rb") as fp:   # Unpickling
    encoderX = pickle.load(fp)

encoderX

[['Cheers', 0],
 ['Drinking', 1],
 ['Opening _ Closing', 2],
 ['Pouring', 3],
 ['Shaking', 4]]

In [84]:
# make the predict
y_load_predict=load_model.predict(X_test)

In [85]:
y_label = []
y_code = y_load_predict.argmax(axis=1)
for code in y_code:
    y_label.append(encoderX[code])

y_label

[['Drinking', 1], ['Shaking', 4], ['Pouring', 3]]